In [ ]:
! pip install transformers

In [1]:
# importing dependencies use to this case study.
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.layers import Dense
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Load IMDB Movie Reviews dataset.

### Source:
https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [2]:
# Loading the IMDB dataset from drive.
imdb_data = pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


###**Text Preprocessing :**

In [3]:
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [4]:
def preprocess_text(sen):

    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z.!]', ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [5]:
reviews = []
for sentence in imdb_data['review'].values:
  reviews.append(preprocess_text(sentence))
reviews = np.array(reviews)  
sentiment = np.array(list(map(lambda a:1 if a=='positive' else 0, imdb_data['sentiment'].values))) # Map positive to 1 and negative to 0

In [6]:
reviews[:5]

array(['One of the other reviewers has mentioned that after watching just Oz episode you ll be hooked. They are right as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO. Trust me this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs sex or violence. Its is hardcore in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda. Em City is home to many..Aryans Muslims gangstas Latinos Christians Italians Irish and more....so scuffles death stares dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dar

## Split dataset: 40000 for train and 10000 for test.

In [7]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(reviews, sentiment, test_size = 0.2, stratify = sentiment, random_state = 42)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(40000,)
(40000,)
(10000,)
(10000,)


In [8]:
del imdb_data, reviews, sentiment # delete data that will not use 

In [9]:
x_train[:5]

array(['I caught this little gem totally by accident back in or . I was at a revival theatre to see two old silly sci fi movies. The theatre was packed full and with no warning they showed a bunch of sci fi short spoofs to get us in the mood . Most were somewhat amusing but THIS came on and within seconds the audience was in hysterics! The biggest laugh came when they showed Princess Laia having huge cinnamon buns instead of hair on her head. She looks at the camera gives a grim smile and nods. That made it even funnier! You gotta see Chewabacca played by what looks like a Muppet! It was extremely silly and stupid...but I couldn t stop laughing. Most of the dialogue was drowned out because of all the laughter. Also if you know Star Wars pretty well it s even funnier they deliberately poke fun at some of the dialogue. This REALLY works with an audience! A definite !',
       'I can t believe that I let myself into this movie to accomplish a favor my friends ask me early this April . Thi

## Load the pretrained Bert model to extract features.

In [10]:
from transformers import DistilBertTokenizer, DistilBertConfig, TFDistilBertModel

In [ ]:
distil_bert = 'distilbert-base-uncased' # Name of the pretrained models : (Wights)

#DistilBERT 
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model = TFDistilBertModel.from_pretrained(distil_bert)

### Some toy examples

In [12]:
e = tokenizer.encode("You'll come back tomorrow. !")
print(e)
print("="*50)
print(tokenizer.decode(e))

[101, 2017, 1005, 2222, 2272, 2067, 4826, 1012, 999, 102]
[CLS] you'll come back tomorrow.! [SEP]


In [13]:
e = tokenizer.encode(x_train[0])
print(e)
print("="*100)
print(tokenizer.decode(e))

[101, 1045, 3236, 2023, 2210, 17070, 6135, 2011, 4926, 2067, 1999, 2030, 1012, 1045, 2001, 2012, 1037, 6308, 3004, 2000, 2156, 2048, 2214, 10021, 16596, 10882, 5691, 1012, 1996, 3004, 2001, 8966, 2440, 1998, 2007, 2053, 5432, 2027, 3662, 1037, 9129, 1997, 16596, 10882, 2460, 11867, 21511, 2015, 2000, 2131, 2149, 1999, 1996, 6888, 1012, 2087, 2020, 5399, 19142, 2021, 2023, 2234, 2006, 1998, 2306, 3823, 1996, 4378, 2001, 1999, 1044, 27268, 22420, 2015, 999, 1996, 5221, 4756, 2234, 2043, 2027, 3662, 4615, 21110, 2050, 2383, 4121, 21229, 21122, 2015, 2612, 1997, 2606, 2006, 2014, 2132, 1012, 2016, 3504, 2012, 1996, 4950, 3957, 1037, 11844, 2868, 1998, 11232, 1012, 2008, 2081, 2009, 2130, 4569, 14862, 999, 2017, 10657, 2156, 21271, 19736, 16665, 2209, 2011, 2054, 3504, 2066, 1037, 14163, 29519, 999, 2009, 2001, 5186, 10021, 1998, 5236, 1012, 1012, 1012, 2021, 1045, 2481, 1056, 2644, 5870, 1012, 2087, 1997, 1996, 7982, 2001, 12805, 2041, 2138, 1997, 2035, 1996, 7239, 1012, 2036, 2065, 2017, 

In [14]:
len(e)

194

In [15]:
output = model(tf.constant(e)[None, :])
print(type(output))
output

<class 'tuple'>


(<tf.Tensor: shape=(1, 194, 768), dtype=float32, numpy=
 array([[[ 0.15895611, -0.12386137,  0.05420507, ..., -0.03344058,
           0.55348414,  0.3435394 ],
         [ 0.52001077, -0.32365903, -0.01601099, ...,  0.00661454,
           0.69623977,  0.25855684],
         [-0.06160792, -0.22191559,  0.34980658, ..., -0.09395681,
           0.01518752, -0.37404358],
         ...,
         [ 0.37388927, -0.07064627,  0.42436817, ..., -0.3785236 ,
           0.39427185, -0.38435003],
         [-0.04955504, -0.14920276,  0.03789302, ...,  0.22821037,
           0.4417211 , -0.4650039 ],
         [ 0.54786795,  0.3371322 ,  0.3964695 , ...,  0.41849664,
           0.20822771, -0.2171463 ]]], dtype=float32)>,)

In [22]:
len(output[0][0][2])

768

In [16]:
max_len = 0
for sent in x_train:
  l = len(sent)
  if l>max_len:
    max_len = l
print(max_len)    

13458


##**Feature Extraction using pretrained Bert :**

In [17]:
model.trainable = False

In [29]:
# Features extraction for train.
X_train = []
for sent in x_train:
  if len(sent) >= 512:
    sent = sent[:510]
  e = tokenizer.encode(sent)
  output = model(tf.constant(e)[None, :])
  X_train.append(output[0][0][0])
X_train = np.array(X_train) 
print(X_train.shape) 

(40000, 768)


In [30]:
np.save('/content/drive/My Drive/X_train_imdb.npy', X_train)

In [33]:
# Feature extraction for test.
from tqdm import tqdm
X_test = []
for sent in tqdm(x_test):
  if len(sent) >= 512:
    sent = sent[:510]
  e = tokenizer.encode(sent)
  output = model(tf.constant(e)[None, :])
  X_test.append(output[0][0][0])
X_test = np.array(X_test) 
print(X_test.shape)
np.save('/content/drive/My Drive/X_test_imdb.npy', X_test)

100%|██████████| 10000/10000 [10:09<00:00, 16.40it/s]


(10000, 768)


In [34]:
X_train = np.load('/content/drive/My Drive/X_train_imdb.npy')
X_test = np.load('/content/drive/My Drive/X_test_imdb.npy')
print(X_train.shape)
print(X_test.shape)

(40000, 768)
(10000, 768)


## Fully connected Neural nets Architecture.

### To train on the features that extracted from pretrained Bert.

In [81]:
input_layer = tf.keras.layers.Input(shape=(768, ))

x = tf.keras.layers.Dense(1024, activation='elu', kernel_initializer='he_normal')(input_layer)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='elu', kernel_initializer='he_normal')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='elu', kernel_initializer='he_normal')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='elu', kernel_initializer='he_normal')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='elu', kernel_initializer='he_normal')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='elu', kernel_initializer='he_normal')(x)

output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

clf_model = tf.keras.models.Model(input_layer, output_layer)

clf_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 768)]             0         
_________________________________________________________________
dense_46 (Dense)             (None, 1024)              787456    
_________________________________________________________________
dropout_33 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_47 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_34 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_35 (Dropout)         (None, 512)               0  

In [82]:
# Compile the model.
clf_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [83]:
# Callbacks.
earlystopper = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/IMDB_review_clf_Model', verbose=1, save_best_only=True)

In [84]:
# Train model.
history = clf_model.fit(X_train,y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test), callbacks=[earlystopper, checkpointer])

Epoch 1/100
304/313 [============================>.] - ETA: 0s - loss: 0.8164 - accuracy: 0.6030
Epoch 00001: val_loss improved from inf to 0.54366, saving model to /content/drive/My Drive/IMDB_review_clf_Model
INFO:tensorflow:Assets written to: /content/drive/My Drive/IMDB_review_clf_Model/assets
313/313 [==============================] - 3s 8ms/step - loss: 0.8091 - accuracy: 0.6065 - val_loss: 0.5437 - val_accuracy: 0.7550
Epoch 2/100
302/313 [===========================>..] - ETA: 0s - loss: 0.5127 - accuracy: 0.7576
Epoch 00002: val_loss improved from 0.54366 to 0.42734, saving model to /content/drive/My Drive/IMDB_review_clf_Model
INFO:tensorflow:Assets written to: /content/drive/My Drive/IMDB_review_clf_Model/assets
313/313 [==============================] - 3s 9ms/step - loss: 0.5118 - accuracy: 0.7579 - val_loss: 0.4273 - val_accuracy: 0.8100
Epoch 3/100
299/313 [===========================>..] - ETA: 0s - loss: 0.4738 - accuracy: 0.7811
Epoch 00003: val_loss did not improve f

In [85]:
# Evaluation.
cls_test_model = tf.keras.models.load_model('/content/drive/My Drive/IMDB_review_clf_Model')
score = cls_test_model.evaluate(X_test, y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.4042 - accuracy: 0.8215


In [86]:
# Prediction for some test datapoints.
for i in range(10,101,10):

  pred = cls_test_model.predict(X_test[i][None, :])
  print('Actual =', y_test[i], 'Predicted =',np.round(pred,2))

Actual = 0 Predicted = [[0.4]]
Actual = 1 Predicted = [[0.85]]
Actual = 0 Predicted = [[0.71]]
Actual = 1 Predicted = [[0.98]]
Actual = 0 Predicted = [[0.1]]
Actual = 1 Predicted = [[0.5]]
Actual = 1 Predicted = [[0.48]]
Actual = 1 Predicted = [[0.6]]
Actual = 0 Predicted = [[0.29]]
Actual = 0 Predicted = [[0.15]]
